# Imports

In [1]:
# import required modules 
import requests, json 

# Pull weather data

Code from https://www.geeksforgeeks.org/python-find-current-weather-of-any-city-using-openweathermap-api/

In [9]:
def get_weather(city_name):
    # Python program to find current  
    # weather details of any city 
    # using openweathermap api 

    # Enter your API key here 
    api_key = "9291be5e6daedb7e3a4a03f8857a8d5f"

    # base_url variable to store url 
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # complete_url variable to store 
    # complete url address 
    complete_url = base_url + "appid=" + api_key + "&q=" + city_name 

    # get method of requests module 
    # return response object 
    response = requests.get(complete_url) 

    # json method of response object  
    # convert json format data into 
    # python format data 
    x = response.json() 
    
    # Now x contains list of nested dictionaries 
    # Check the value of "cod" key is equal to 
    # "404", means city is found otherwise, 
    # city is not found 
    if x["cod"] != "404": 
        return x
    else: 
        print(" City Not Found ") 
        return None

In [10]:
get_weather("Austin")

{'coord': {'lon': -97.74, 'lat': 30.27},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 288.57,
  'feels_like': 285.52,
  'temp_min': 287.59,
  'temp_max': 289.26,
  'pressure': 1021,
  'humidity': 48},
 'visibility': 10000,
 'wind': {'speed': 2.6, 'deg': 0},
 'clouds': {'all': 90},
 'dt': 1603564134,
 'sys': {'type': 1,
  'id': 3344,
  'country': 'US',
  'sunrise': 1603543170,
  'sunset': 1603583423},
 'timezone': -18000,
 'id': 4671654,
 'name': 'Austin',
 'cod': 200}